In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
alljobssoups = []
alljoburls = []
alljoburlstems = []

#Determine how many pages of ads are on https://jobs.chronicle.com/jobs
#When collected, was 665, now 852
#For up-to-minute, use RSS feed

for i in range(1, 665):
    #print(i)
    page=requests.get('https://jobs.chronicle.com/jobs/'+str(i)+'/')
    soup = BeautifulSoup(page.text, 'html.parser')
    alljobssoups.append(soup)\
    for a in soup.find_all('a', attrs={'class': 'js-clickable-area-link'}):
        alljoburls.append('https://jobs.chronicle.com'+str(a.get('href').replace(" \r\n\t", "").replace("\r\n\r\n\r\n\r\n", "")))
        alljoburlstems.append(a.get('href').replace(" \r\n\t", "").replace("\r\n\r\n\r\n\r\n", ""))
    print(str(i)+' is done. '+str(664-i)+' are left.')
    time.sleep(0.5)

In [ ]:
alldescsoups = []
alldesctitle = []
employ = []
loc = []
salary = []
date = []
categories = []
level = []
employtype = []
text = []

In [ ]:
#Check length of alljoburls list for range
for i in range(0,13267):
    page = requests.get(alljoburls[i])
    soup = BeautifulSoup(page.text, 'html.parser')
    alldescsoups.append(soup)
    #number of 'dd' tags determines number of job info fields. 7 is max available fields
    if len(soup.find_all('dd')) == 7:
        employ.append(soup.find_all('dd')[0].get_text().replace("\n", ""))
        loc.append(soup.find_all('dd')[1].get_text().replace("\n", ""))
        salary.append(soup.find_all('dd')[2].get_text().replace("\n", ""))
        date.append(soup.find_all('dd')[3].get_text().replace("\n", ""))
        categories.append(soup.find_all('dd')[4].get_text().replace("\r", "").replace("\t", "").replace(", \n\n", "; ").replace("\n", ""))
        level.append(soup.find_all('dd')[5].get_text().replace("\n", ""))
        employtype.append(soup.find_all('dd')[6].get_text().replace("\n", ""))
        text.append(soup.find_all(attrs = {'class': 'block fix-text job-description'}))
        for a in soup.find_all('h1'):
            alldesctitle.append(a.get_text())
    #'dd' == 6 is when the "level" field is missing, which is usually for admin positions
    elif len(soup.find_all('dd')) == 6:
        employ.append(soup.find_all('dd')[0].get_text().replace("\n", ""))
        loc.append(soup.find_all('dd')[1].get_text().replace("\n", ""))
        salary.append(soup.find_all('dd')[2].get_text().replace("\n", ""))
        date.append(soup.find_all('dd')[3].get_text().replace("\n", ""))
        categories.append(soup.find_all('dd')[4].get_text().replace("\r", "").replace("\t", "").replace(", \n\n", "; ").replace("\n", ""))
        level.append('Not Applicable')
        employtype.append(soup.find_all('dd')[5].get_text().replace("\n", ""))
        text.append(soup.find_all(attrs = {'class': 'block fix-text job-description'}))
        for a in soup.find_all('h1'):
            alldesctitle.append(a.get_text())
    #Any other cases are treated as an error
    else:
        employ.append('error')
        loc.append('error')
        salary.append('error')
        date.append('error')
        categories.append('error')
        level.append('error')
        employtype.append('error')
        text.append(soup.find_all(attrs = {'class': 'block fix-text job-description'}))
        for a in soup.find_all('h1'):
            alldesctitle.append(a.get_text())
    #Progress statement
    print(str(i+1)+' is done.'+str(13267-i+1)+' remain.')
    #Delayed to not overly tax server
    time.sleep(0.5)

In [ ]:
#Build one DataFrame from all lists. Will throw an error here if something is wrong.
alljobdat = pd.DataFrame({'title': alldesctitle, 'employer': employ, 'location': loc, 'salary': salary, 'post_date': date, 'categories': categories, 'pos_type': level, 'employ_type': employtype, 'description': text, 'url': alljoburls})
#Export CSV with data
alljobdat.to_csv('chronicle_jobs_utf8.csv', index=False, encoding='utf-8')

In [2]:
#Read in data collected Nov 15 2020
dat = pd.read_csv(r"H:\Dropbox\Kent\FA20\Doc Review\data files\chronicle_jobs_utf8.csv")

In [3]:
dat.keys()

Index(['title', 'employer', 'location', 'salary', 'post_date', 'categories',
       'pos_type', 'employ_type', 'description', 'url'],
      dtype='object')

In [4]:
dat.count()

title          13267
employer       13267
location       13267
salary         13267
post_date      13267
categories     13267
pos_type       13267
employ_type    13267
description    13267
url            13267
dtype: int64

In [5]:
dat['post_date'][0]

'Oct 30, 2020'

In [6]:
dat['post_date'][-1:]

13266    Aug 13, 2019
Name: post_date, dtype: object

In [7]:
dat['categories'].head()

0    Faculty Positions; Science, Technology & Mathe...
1    Faculty Positions; Science, Technology & Mathe...
2    Faculty Positions; Education; Education Admini...
3    Faculty Positions; Health & Medicine; Medicine...
4    Faculty Positions; Arts; Performing Arts; Jobs...
Name: categories, dtype: object

In [8]:
dat['tag_list'] = [x.split(";") for x in dat['categories']]

In [9]:
for i in range(13267-1):
    dat['tag_list'][i] = [x.lstrip() for x in  dat['tag_list'][i]]

In [10]:
all_tags = []

In [11]:
for i in dat['tag_list']:
    for x in i:
        if x not in all_tags:
            all_tags.append(x)

In [12]:
all_tags

['Faculty Positions',
 'Science, Technology & Mathematics',
 'Engineering',
 'Education',
 'Education Administration & Leadership',
 'Health & Medicine',
 'Medicine',
 'Other Health & Medicine',
 'Arts',
 'Performing Arts',
 'Jobs Outside Academe',
 'Non-profit & Government Organizations',
 'Administrative',
 'Business & Administrative Affairs',
 'Chief Business Officers & Vice Presidents',
 'Other Business & Administrative Affairs',
 'Academic Affairs',
 'Other Academic Affairs',
 'Alumni Affairs',
 'Development & Advancement',
 'Social & Behavioral Sciences',
 'Psychology',
 'Deans',
 'Public Relations & Marketing (Campus)',
 'Auxiliary Services',
 'Librarians & Library Administration',
 'Technology Administration/Other',
 'Financial Affairs',
 'Counsellor Education',
 'Curriculum & Instruction',
 'Nursing',
 'Humanities',
 'History',
 'Political Science & International Relations',
 'Other Social & Behavioral Sciences',
 'Professional Fields',
 'Vocational & Technical Fields',
 'Stud

In [13]:
len(all_tags)

172

In [14]:
irr_tags = ['Engineering',
 'Medicine',
 'Business & Administrative Affairs',
 'Chief Business Officers & Vice Presidents',
 'Other Business & Administrative Affairs',
 'Academic Affairs',
 'Other Academic Affairs',
 'Alumni Affairs',
 'Development & Advancement',
 'Deans',
 'Auxiliary Services',
 'Financial Affairs',
 'Counsellor Education',
 'Nursing',
 'History',
 'Vocational & Technical Fields',
 'Student Affairs',
 'Athletics',
 'Kinesiology, Exercise Physiology & Physical Education',
 'Biology & Life Sciences',
 'Surgery',
 'Economics',
 'Business & Administrative Support',
 'Executive',
 'Executive Directors',
 'Mathematics',
 'Chief Academic Officers & Vice Presidents',
 'Music',
 'Academic Advising & Academic Support Services',
 'Chief Student Affairs Officers & Vice Presidents',
 'Management',
 'Special Education',
 'Other Student Affairs',
 'Pediatrics',
 'Other Executive',
 'Health Services',
 'Financial Aid',
 'Facilities Management & Physical Plant',
 'Disabled Student Services',
 'Student Activities & Services',
 'Minority & Multicultural Affairs',
 'Accounting & Finance',
 'Business Administration',
 'Classics',
 'Faculty Affairs',
 'Security Studies',
 'Veterinary Sciences',
 'Agriculture & Animal Sciences',
 'Legal Affairs',
 'Design & Graphic Arts',
 'Conference & Special Event Administration',
 'Criminal Justice & Criminology',
 'Extension Services',
 'Residence Life',
 'Dentistry',
 'Physical Therapy',
 'Chancellors & Presidents',
 'Anthropology & Archaeology',
 'Physics & Space Sciences',
 'Chemistry & Biochemistry',
 'Nutrition',
 'Biotechnology & Bioengineering',
 'Art History',
 'Pharmacology',
 'Architecture',
 'Geology & Earth Sciences',
 'Occupational Therapy',
 'Speech & Hearing Sciences',
 'Food Services Management',
 'Provosts',
 'Educational Service Providers',
 'Geography',
 'Primary Care',
 'Principals & Headmasters',
 'Religious Affairs & Ministry',
 'Superintendents & Commissioners']

In [15]:
len(irr_tags)

76

In [16]:
rel_tags = [x for x in all_tags if x not in irr_tags]
rel_tags

['Faculty Positions',
 'Science, Technology & Mathematics',
 'Education',
 'Education Administration & Leadership',
 'Health & Medicine',
 'Other Health & Medicine',
 'Arts',
 'Performing Arts',
 'Jobs Outside Academe',
 'Non-profit & Government Organizations',
 'Administrative',
 'Social & Behavioral Sciences',
 'Psychology',
 'Public Relations & Marketing (Campus)',
 'Librarians & Library Administration',
 'Technology Administration/Other',
 'Curriculum & Instruction',
 'Humanities',
 'Political Science & International Relations',
 'Other Social & Behavioral Sciences',
 'Professional Fields',
 'Business & Management',
 'Other Business & Management',
 'Other Education',
 'Computer Sciences & Technology',
 'Public Administration & Policy',
 'Computer Services & Information Technology',
 'Marketing & Sales',
 'Grants & Contracts',
 'Statistics',
 'Communications',
 'Other Communications',
 'Social Work & Human Services',
 'Admissions, Enrollment, Retention & Registrar',
 'Academic Admin

In [17]:
len(rel_tags)

96

In [18]:
dat['tag_list'][0]

['Faculty Positions', 'Science, Technology & Mathematics', 'Engineering']

In [19]:
dat.iloc[1]

title          Assistant Professor, Tenure Track, in Biomedic...
employer                                University of Washington
location                               Washington, United States
salary                                      Salary Not specified
post_date                                           Oct 30, 2020
categories     Faculty Positions; Science, Technology & Mathe...
pos_type                                   Tenured/Tenured Track
employ_type                                            Full Time
description    [<div class="block fix-text job-description">\...
url            https://jobs.chronicle.com/job/297333/assistan...
tag_list       [Faculty Positions, Science, Technology & Math...
Name: 1, dtype: object

In [20]:
reldat = pd.DataFrame(columns = dat.keys())
reldat

,title,employer,location,salary,post_date,categories,pos_type,employ_type,description,url,tag_list


In [21]:
bool(set(dat['tag_list'][1]).isdisjoint(irr_tags))

False

In [130]:
dat['tag_list'][12]

['Administrative', ' Academic Affairs', ' Librarians & Library Administration']

In [22]:
for i in range(13267-1):
    if(set(dat['tag_list'][i]) & set(irr_tags)):
        pass
    else:
        reldat = reldat.append(dat.iloc[i], ignore_index = True)

In [23]:
reldat.head()

,title,employer,location,salary,post_date,categories,pos_type,employ_type,description,url,tag_list
0,"ASSOCIATE VICE PRESIDENT, TEACHING AND LEARNING",Kwantlen Polytechnic University,Canada,Competitive Salary,"Oct 30, 2020",Faculty Positions; Education; Education Admini...,Executive,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/297332/associat...,"[Faculty Positions, Education, Education Admin..."
1,Assistant Vice President of Faculty and Creati...,New Jersey Performing Arts Center,"New Jersey, United States",Salary Not specified,"Oct 30, 2020",Faculty Positions; Arts; Performing Arts; Jobs...,Executive,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/297330/assistan...,"[Faculty Positions, Arts, Performing Arts, Job..."
2,Associate or Full Professor / Chair,The University of Texas at Tyler,"Texas, United States",Salary Commensurate with experience,"Oct 30, 2020",Faculty Positions; Social & Behavioral Science...,Tenured/Tenured Track,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/297325/associat...,"[Faculty Positions, Social & Behavioral Scienc..."
3,Assistant Professor - MSW Program,Carlow University,"Pennsylvania, United States",Competitive Salary,"Oct 30, 2020",Faculty Positions; Social & Behavioral Science...,Tenured/Tenured Track,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/297315/assistan...,"[Faculty Positions, Social & Behavioral Scienc..."
4,"Assistant Professor, School Psychology",State University Of New York College at Platts...,"New York, United States",Salary Not specified,"Oct 30, 2020",Faculty Positions; Social & Behavioral Science...,Non-Tenured Track,Full Time,"[<div class=""block fix-text job-description"">\...",https://jobs.chronicle.com/job/297314/assistan...,"[Faculty Positions, Social & Behavioral Scienc..."


In [24]:
sum(1 for x in reldat['tag_list'] if 'Faculty Positions' in x)

2052

In [26]:
sum(1 for x in reldat['tag_list'] if 'Social & Behavioral Sciences' in x)

367